In [1]:
#Let's try to make these calls faster
import tekore as tk
import asyncio
import yaml
import requests


In [2]:
credentials_path = "credentials.yaml"
with open(credentials_path) as file:
    cred = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
conf = cred['client_id'], cred['client_secret']

app_token = tk.request_client_token(*conf)

#Let's try to connect these words
spotify = tk.Spotify(app_token)


In [4]:
app_token.access_token

'BQAY8_-0-Fm2-Adz0m8_RpBU4GUrG2BoR1QeRpT6_W9aO91p92QTTK7Q_zQa_xV8rY2dk-Je2pu0FIbPFR0'

## Playlist search

In [5]:
obj = spotify.search("Top 50 USA", types=("playlist",), limit=1)

In [6]:
ids = []
for track in spotify.playlist_items(obj[0].items[0].id, as_tracks=True)['items']:
    ids.append(track["track"]["id"])


In [7]:
features = spotify.tracks_audio_features(ids)

In [8]:
type(features)

tekore.model.ModelList

## Track search

In [9]:
obj = spotify.search("Show must go on", types=("track",), limit=1)

In [10]:
obj[0].items[0].id

'3m86qx7aIeqFcYcQ1UXUvP'

In [11]:
features = spotify.track_audio_features(obj[0].items[0].id)

In [107]:
features

AudioFeatures with fields:
  acousticness = 0.526
  analysis_url = 'https://api.spotify.com/v1/audio-analysis/3m86qx7aIeq...'
  danceability = 0.421
  duration_ms = 272107
  energy = 0.517
  id = '3m86qx7aIeqFcYcQ1UXUvP'
  instrumentalness = 1.82e-06
  key = 11
  liveness = 0.136
  loudness = -11.49
  mode = 0
  speechiness = 0.035
  tempo = 84.011
  time_signature = 4
  track_href = 'https://api.spotify.com/v1/tracks/3m86qx7aIeqFcYcQ1UXUvP'
  type = 'audio_features'
  uri = 'spotify:track:3m86qx7aIeqFcYcQ1UXUvP'
  valence = 0.109

In [28]:
spotify.track_audio_features('3m86qx7aIeqFcYcQ1UXUvP')

AudioFeatures with fields:
  acousticness = 0.526
  analysis_url = 'https://api.spotify.com/v1/audio-analysis/3m86qx7aIeq...'
  danceability = 0.421
  duration_ms = 272107
  energy = 0.517
  id = '3m86qx7aIeqFcYcQ1UXUvP'
  instrumentalness = 1.82e-06
  key = 11
  liveness = 0.136
  loudness = -11.49
  mode = 0
  speechiness = 0.035
  tempo = 84.011
  time_signature = 4
  track_href = 'https://api.spotify.com/v1/tracks/3m86qx7aIeqFcYcQ1UXUvP'
  type = 'audio_features'
  uri = 'spotify:track:3m86qx7aIeqFcYcQ1UXUvP'
  valence = 0.109

### Dummy dataset

In [113]:
import sys
sys.path.append("app")
import pandas as pd
%load_ext autoreload

%autoreload 2

from spotifyData import getdata_faster
from utils import request_manager
creator = getdata_faster.DatasetCreator(spotify_api=spotify)

request_manager = request_manager.RequestManager()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [134]:

dfs = []
phrases = ["Top 50 Poland", "Top 50 USA", "Top 50 belgium", "Top 50 Japan", "Top 50 Australia", "Top 50 Egypt"]
for phrase in phrases:
    outcome = creator.search(phrase, type="playlist")
    rows = request_manager.add_data([], outcome, query_name=phrase)
    dfs.append(pd.DataFrame(rows))


In [135]:
df = pd.concat(dfs, axis=0)

In [137]:
df.to_csv("Different_top_50.csv")